# Ethereum Contracts

This notebook demonstrates how to use the `ethereum_contracts` dataset to answer questions about the history of contracts on Ethereum.

Additional information about this dataset can be found [here](https://github.com/paradigmxyz/paradigm-data-portal/tree/main/datasets/ethereum_contracts)

The examples below use [polars](https://github.com/pola-rs/polars) library to read the files directly without needing to import them to a database. Files are stored in S3 and are loaded using `pl.scan_arrows_dataset`. It is recommended to tuse the 'large' instance with this config.


## Ethereum contracts by the numbers

All of these fun facts are computed in the examples below:
```
- 20% of all ethereum contracts have been deployed by just 10 EOAs
- 49% of all ethereum contracts have been created by just 10 factories
- 56% of all ethereum contracts use just 10 bytecode templates
- 0.46% of ethereum addresses have been deployed more than once
- The most times that a single contract address has been deployed is 848
- The most contracts deployed in a single transaction or block is 701
- 57M contracts have been deployed by 4.2M EOAs using 492K factories
- Only 878K unique contracts have been deployed
- The most leading zeros in a contract address is 14, of which there are 2 contracts
```

In [2]:
import math
import os
import shutil
import subprocess

import ctc
import ctc.rpc
import polars as pl
import matplotlib.pyplot as plt
import toolplot
import numpy as np
import toolstr
import s3fs
import boto3
import sagemaker
import pyarrow.dataset as ds
fs = s3fs.S3FileSystem()

sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /root/.config/sagemaker/config.yaml


In [3]:
# Set SageMaker and S3 client variables
sess = sagemaker.Session()

region = sess.boto_region_name
s3_client = boto3.client("s3", region_name=region)

sagemaker_role = sagemaker.get_execution_role()

# Set read and write S3 buckets and locations
data_bucket = sess.default_bucket()

sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /root/.config/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /root/.config/sagemaker/config.yaml


In [4]:
# set up env

dataset = "ethereum_contracts"
data_path = f"{data_bucket}/{dataset}"

# Get a list of all files in the directory
files = fs.ls(data_path)

# Filter the list to include only Parquet files
parquet_files = [f for f in files if f.endswith('.parquet')]

dataset = ds.dataset(parquet_files, format='parquet', filesystem=fs)
df = pl.scan_pyarrow_dataset(dataset)

# Let's count the entites and events in the data...

In [6]:
def get_unique_column_count(column):
    result = df.select(
        pl.col(column).n_unique(),
    ).collect(streaming=True)
    return result[column][0]

### How many contract deployments have there been?

In [5]:
result = df.select(pl.col('create_index').count()).collect()
n_deployments = result['create_index'][0]

n_deployments

57887821

### How many unique contract addresses have been deployed?

In [13]:
n_unique_contract_addresses = get_unique_column_count('contract_address')

n_unique_contract_addresses

57314368

### How many factories have been used to create contracts?

In [14]:
get_unique_column_count('factory')

491663

### How many EOAs have deployed contracts?

In [7]:
get_unique_column_count('deployer')

4228306

### How many unique contract bytecodes have been deployed?

In [8]:
get_unique_column_count('code_hash')

878235

### How many contract addresses have been deployed more than one time?

In [15]:
n_deployed_more_than_once = (
    df
    .groupby(pl.col("contract_address"))
    .agg(pl.count())
    .filter(pl.col('count') > 1)
    .select(pl.count())
    .collect(streaming=True)
).rows()[0][0]

n_deployed_more_than_once

/tmp/ipykernel_714/328330587.py:3: DeprecationWarning: `groupby` is deprecated. It has been renamed to `group_by`.
  .groupby(pl.col("contract_address"))


265431

In [16]:
more_than_once_percent = toolstr.format(
    n_deployed_more_than_once / n_unique_contract_addresses,
    percentage=True,
    decimals=2,
)
print(more_than_once_percent, 'of contract addresses have been deployed more than once')

0.46% of contract addresses have been deployed more than once


# Let's look at some top 10 leaderboards

In [17]:
def get_top_n(column, n):
    col = '0x' + (pl.col(column).struct.field(column).bin.encode('hex'))
    return (
        df
        .select(pl.col(column).value_counts())
        .select([col.alias(column), pl.col(column).struct.field('counts')])
        .sort('counts', descending=True)
        .head(n)
        .collect(streaming=True)
    )

### Which EOAs have deployed the most contracts?

In [18]:
top_10_deployers = get_top_n('deployer', 10)

top_10_deployers

deployer,counts
str,u32
"""0xffff46e05a09…",2854163
"""0xe5c99b740572…",1822822
"""0x6cace0528324…",1579922
"""0x5f5a901463fb…",954224
"""0xf24c621e5108…",952859
"""0x9901c66f2d4b…",801294
"""0xfc9161b4ca1e…",665752
"""0x2aab3ddd93ed…",600176
"""0x06ca51ab3c82…",542637


In [13]:
top_10_deployer_percent = toolstr.format(
    top_10_deployers['counts'].sum() / n_deployments,
    percentage=True,
    decimals=1,
)

print('The top 10 deployers have deployed', top_10_deployer_percent, 'of all contracts on Ethereum')

The top 10 deployers have deployed 19.5% of all contracts on Ethereum


### Which factories have deployed the most contracts?

In [14]:
top_10_factories = get_top_n('factory', 10)

top_10_factories

factory,counts
str,u32
"""0x0000000000004946c0e9f43f4dee607b0ef1fa1c""",10364776
"""0x0000000000b3f879cb30fe243b4dfee438691c04""",6493824
"""0x0de8bf93da2f7eecb3d9169422413a9bef4ef628""",2324361
"""0x0000000000007f150bd6f54c40a34d7c3d5e9f56""",1647068
"""0xa3c1e324ca1ce40db73ed6026c4a177f099b5770""",1586334
"""0xa5409ec958c83c3f309868babaca7c86dcb077c1""",1552418
"""0x8a91c9a16cd62693649d80afa85a09dbbdcb8508""",1088378
"""0x0a252663dbcc0b073063d6420a40319e438cfa59""",1021675
"""0x881d4032abe4188e2237efcd27ab435e81fc6bb1""",994275


In [15]:
top_10_factory_percent = toolstr.format(
    top_10_factories['counts'].sum() / n_deployments,
    percentage=True,
    decimals=1,
)

print('The top 10 factories have deployed', top_10_factory_percent, 'of all contracts on Ethereum')

The top 10 factories have deployed 48.4% of all contracts on Ethereum


### Which contract bytecodes are most common?

In [16]:
top_10_bytecodes = get_top_n("code_hash", 10)

top_10_bytecodes

code_hash,counts
str,u32
"""0x1d93f60f105899172f7255c030301c3af4564edd4a48577dbdc448aec7ddb0ac""",10365216
"""0xd80cd839dd3957d572b90780ada202a13936fa2875daea94216263371e9ef1d2""",6494337
"""0xc5d2460186f7233c927e7db2dcc703c0e500b653ca82273b7bfad8045d85a470""",4853969
"""0x562d59a51820d47f520c975e0b2bcffac644a509749a3161f481f57b6e826d21""",2324361
"""0x8f789d24f75f80df63f1d69c18963fb4c2dea81ade3b780fd0c886e4d1fabc4a""",1647068
"""0xce33220d5c7f0d09d75ceff76c05863c5e7d6e801c70dfe7d5d45d4c44e80654""",1595657
"""0x99c99f1a6d65a9097e0f8ca61683878ed26099e347359f5cdc600bd194dbe908""",1548789
"""0x647d5d7eb5af6fbf9ed95320cadf658cec3819ffa536adc7b6387fdf1b72123d""",1542104
"""0x2df39c0062882ba634fb24c89e7f9871ecef10a6e6a6e79a336f6ed463d5c942""",1175582


In [17]:
top_10_bytecode_percent = toolstr.format(
    top_10_bytecodes['counts'].sum() / n_deployments,
    percentage=True,
    decimals=1,
)

print('The top 10 bytecodes represent', top_10_bytecode_percent, 'of all contracts on Ethereum')

The top 10 bytecodes represent 56.4% of all contracts on Ethereum


### Which contract addresses have been deployed the most number of times?

In [18]:
top_10_contract_addresses = get_top_n('contract_address', 10)

top_10_contract_addresses

contract_address,counts
str,u32
"""0xba4ac7aadfa00003a20c954e077d5c81994c8ece""",848
"""0x9c14d5cb81d576e97874b593bffdee670347bfd6""",843
"""0xa46ec029df976770a75b2ce42fd4276ddc26a961""",818
"""0x08bd3b769ffd7992e1d14101a2235eeff7232c57""",817
"""0x3e2d8eabd99bc994d2ab4a804c6f804120af123e""",743
"""0x71664ea68d994fb064aed08807257b2c6dfbd65d""",730
"""0x472875a6fde6cc05b53e58f7e7064d5add45c4bf""",729
"""0x8a454242940937c95dffb7b038fe27b61817f036""",497
"""0x19486d3a1da8cfeb4cd19c9c4fe5624a6587673c""",496


### What is the most contracts deployed in a single transaction?

In [19]:
top_10_transactions = get_top_n('transaction_hash', 10)

top_10_transactions

transaction_hash,counts
str,u32
"""0x81ab1942007f36adf8a8b5930e1b0e5e9f9880dca65e972a43b13be51cfe4ad4""",701
"""0x46e7884e8a8d6663135d36a80ab0a9ee2bd286e6f82e4a359c7a2a7841224298""",701
"""0x394937bc18f92f976faba76bf816b9da53d3708d14c71423ec490bbb7820b627""",700
"""0x0fab831e2808129816a241472e64f9b680d6035326e696391c28acea4eb908ba""",700
"""0xc92a2a570ee0048da8779a75ac2c8b98d1d0af49683aa92e46bb844899be1dc9""",700
"""0x00a1f4b87db8f9e16f3912a8099580a6293805a95fb45f0d8d8ab50b685a8daa""",700
"""0xeb5b8ead45667b9d1e0fbbf0eefec4122235afacee05aace04ca71b9b2f501ed""",700
"""0x809ffb6ef139520603e4a5d620a9e501e5a885486bb3e6b1ddc8b49fe759cbfe""",700
"""0x044eae45065067a8dc42280ed50bbbf00537bc8a68de2e516e0206a28bab1878""",700


### Which contracts have the most leading zeros?

In [19]:
sorted_contracts = (
    df
    .select(pl.col('contract_address'))
    .sort('contract_address')
    .unique()
    .head(10)
    .collect(streaming=True)
)['contract_address']

for binary_address in sorted_contracts:
    address = ctc.binary_convert(binary_address, 'prefix_hex')
    print(address, 40 - len(address.lstrip('0x')))

0x089221e9611f5bf6f1ba215c10a1b28969c4bc6a 1
0x25c74196404c932a6641f0e0d8d10775ef8c106f 0
0x4bc7da322a54d6e9dcd44bc50a7c987037ef43c7 0
0x4fce0c814754465bb4b4b70113b8604fbcdbbedd 0
0x736e1b4fb2a78ffd65f68f6cf9d001cd954a2d5c 0
0x93031f936388a58d14ca84607833475d7b930168 0
0xbcf83d4a5965aa194b4acc0495e35321e27f9ed6 0
0xcfe6ea39ce4920143fe5b07213ba0343033fa62a 0
0x07aec74539d94f9072a345cc4a0b6073e2073ef4 1
0xaedc4145e69395001460eaf9bc8fb36a41f02428 0


# We can also perform targeted queries

### Who deployed this contract? When? What was the creation transaction?

In [20]:
DAI = '0x6b175474e89094c44da98b954eedeac495271d0f'

In [ ]:
binary_address = ctc.binary_convert(DAI, 'binary')

result = (
    df
    .filter(pl.col('contract_address') == binary_address)
    .collect(streaming=True)
)
pl_utils.binary_columns_to_prefix_hex(result)

### What are all of the deployed Uniswap V2 pools?

In [6]:
example_pool = '0xb4e16d0168e52d35cacd2c6185b44281ec28c9dc'

In [ ]:
example_bytecode = await ctc.rpc.async_eth_get_code(example_pool)
example_code_hash = ctc.keccak(example_bytecode, output_format='binary')

result = (
    df
    .filter(pl.col('code_hash') == example_code_hash)
    .collect(streaming=True)
)
pl_utils.binary_columns_to_prefix_hex(result)